# Learning by Doing: Dynamic Programming, NumPy, and Levenshtein Distance

## Introduction

You may recall _recursion_ from unit 2: you can structure functions or programs to call themselves, iteratively breaking down a task into smaller subtasks until it reaches a _base case_. You may also recall that in many cases, especially when using Python, recursion is actually not terribly efficient in practice.

One way to think about _dynamic programming_ is as the inverse of recursion: instead of a function calling itself, it builds outwards from simpler subproblems, using information that is already known to make the next step simpler. To preserve this information, dynamic programming uses a technique called _memoization_ (**not** memorization), where information from branches or cases that have already been tested is stored.

To introduce dynamic programming (DP), this notebook presents exercises to implement **[Levenshtein Distance](https://en.wikipedia.org/wiki/Levenshtein_distance)**, a classic algorithm using DP techniques and one with multiple uses in both humanistic and computer-scientific research. Along the way, we will introduce NumPy, a powerful and widely-used library for manipulating mathematical objects.

Levenshtein distance attempts to numerically represent the number of edits it takes to convert one string for another. An edit may be an addition of a letter, a removal of a letter, or replacing one letter with another. For instance, the Levenshtein distance between _cat_ and _cats_ is 1, because you need to add one letter to get the second word. The distance between _cat_ and _cot_ is also one, because you need to replace one letter in the first word to get the second. The distance between _cat_ and _cots_ is two, because you need to replace one letter, then add another, to get the second word.

Below we will implement a simplified form of Levenshtein distance where add, delete, and substitute all cost 1. There are more elaborate forms you can read about on the above Wikipedia article or implment with packages like `Levenshtein`

## Numpy

In order to set up memoization for Levenshtein distance, we will use the NumPy package. Install this using `pip` or `conda` and load it with the following:

In [1]:
import numpy as np

NumPy has many functions, but we will primarily use it for manipulati$ng _arrays_. Programmatically, arrays behave like multi-dimensional lists of floats. Mathematically, they may represent tables, matrices, vectors, and more; as you can probably guess, data structures like these are heavily used in machine learning.

We initiate an array with the `np.array` function. This takes a list or tuple, which may be a list of lists or list of lists. Let's make one corresponding to the matrix:

$ \begin{bmatrix}
 2&4  \\
 6&8  \\
 10&12  \\
\end{bmatrix}$

In [3]:
test_array = np.array([
    [2, 4],
    [6, 8],
    [10, 12]
])

print(test_array)

[[ 2  4]
 [ 6  8]
 [10 12]]


Items in arrays are accessed by their coordinates in the row, column format. These are specifically listed as two (or more) numbers in [square brackets], each placed separately after the array's name. You can also interpret this first deciding on which sublist, then which element of that sublist, of the above structure.

Let's see how to access 6, on the second row of the first column:

In [5]:
print(test_array[1][0])

6


To get the dimensions of an array, you can use `arr.shape`. This returns a tuple object containing the number of rows, then the number of columns (or the number of sublists and the number of elements of each sublist)

In [6]:
print(test_array.shape)

(3, 2)


NumPy includes several ways to quickly generate arrays. These include ways to create arrays of a given size pre-filled with 0s or 1s. Their first argument is a tuple representing their dimensionality

In [8]:
zeros = np.zeros((3, 2))
print(zeros)

[[0. 0.]
 [0. 0.]
 [0. 0.]]


In [11]:
ones = np.ones((4, 5, 2))
print(ones)

[[[1. 1.]
  [1. 1.]
  [1. 1.]
  [1. 1.]
  [1. 1.]]

 [[1. 1.]
  [1. 1.]
  [1. 1.]
  [1. 1.]
  [1. 1.]]

 [[1. 1.]
  [1. 1.]
  [1. 1.]
  [1. 1.]
  [1. 1.]]

 [[1. 1.]
  [1. 1.]
  [1. 1.]
  [1. 1.]
  [1. 1.]]]


## Setting up Levenshtein distance

Let's begin by proceeding step-by-step to implement Levenshtein distance. For now, we will focus on the comparison of two known strings, "Saturday" and "Sunday." Let's initialize them to begin:

In [12]:
str1 = "Saturday"
str2 = "Sunday"

For the purpose of memoization, we will want to initialize a 2D array whose dimensions are the length of string 1 +1 and length of string 2 + 1. 

Why do we want to include the +1? Because the topmost row and leftmost column of the table will represent how many steps it takes to make a blank string equivalent to the string on the left or top so far. This serves as a kind of "base case" or "maximum cost" against which we will weigh other moves.

In the code below, initialize an array of zeroes named `d` (for **d**istances) with the specified properties:

In [20]:
#Your code here

d = np.zeros((len(str1)+1, len(str2)+1))

In [14]:
print(d)

[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]


In line with the above, we want to fill in the topmost row and leftmost column of the table with 0 through the length of "Sunday" and the length of "Saturday" respectively. As a reminder, this serves to represent the "maximum" number of moves it will take, turning an empty string into the portion of "Sunday" or "Saturday" covered so far.

Out of convention we will use `i` for the row number and `j` for the column number.

In [21]:
for i in range(0, len(str1)+1):
        d[i][0] = i
for j in range(0, len(str2)+1):
    d[0][j] = j
    
print(d)

[[0. 1. 2. 3. 4. 5. 6.]
 [1. 0. 0. 0. 0. 0. 0.]
 [2. 0. 0. 0. 0. 0. 0.]
 [3. 0. 0. 0. 0. 0. 0.]
 [4. 0. 0. 0. 0. 0. 0.]
 [5. 0. 0. 0. 0. 0. 0.]
 [6. 0. 0. 0. 0. 0. 0.]
 [7. 0. 0. 0. 0. 0. 0.]
 [8. 0. 0. 0. 0. 0. 0.]]


In the next part, we will proceed character-by-character through both strings using a nested loop. Before we do so, however, we will want to write some code to calculate the **cost** to convert between two characters. Specifically, this should return 0 if they are the same and 1 if they are not. Finish the method `calculate_cost` below:

In [17]:
def calculate_cost(chr1, chr2):
    if chr1 == chr2:
        return 0
    else:
        return 1

Now let's build the loop. We'll want to compare the number of steps it takes to get from each substring of `Saturday` to each substring in `Sunday`. Luckily, we can use our knowledge of the previous cases, stored in `d`, so we only need to check the cells immediately above, to the left, and to the upper left. The cost of moving up or to the left would be 1, as this is equivalent to an insertion or deletion, so we add that to the score there. If the two letters require a substitution, the cost will be 1 but it will move to the upper left. If the two letters are identical, it will cost 0 and move to the upper left. We can use the `min` function to calculate this for us.

Below is an implementation of the loop. It prints `d` after each run through the outer loop, so you can see `d` take shape:

In [22]:
for i in range(1, len(str1)+1):
    for j in range(1, len(str2)+1):
        subs_cost = calculate_cost(str1[i-1], str2[j-1])
        
        d[i][j] = min(
            d[i-1][j] + 1, #str2 deletes relative to str1
            d[i][j-1] + 1, #str2 adds relative to str1
            d[i-1][j-1] + subs_cost #substitution OR equivalency: always chooses equivalency if possible
        )
    print(f"\nFor {str1} up to {str1[:i]}: Got the following values:")
    print(d)


For Saturday up to S: Got the following values:
[[0. 1. 2. 3. 4. 5. 6.]
 [1. 0. 1. 2. 3. 4. 5.]
 [2. 0. 0. 0. 0. 0. 0.]
 [3. 0. 0. 0. 0. 0. 0.]
 [4. 0. 0. 0. 0. 0. 0.]
 [5. 0. 0. 0. 0. 0. 0.]
 [6. 0. 0. 0. 0. 0. 0.]
 [7. 0. 0. 0. 0. 0. 0.]
 [8. 0. 0. 0. 0. 0. 0.]]

For Saturday up to Sa: Got the following values:
[[0. 1. 2. 3. 4. 5. 6.]
 [1. 0. 1. 2. 3. 4. 5.]
 [2. 1. 1. 2. 3. 3. 4.]
 [3. 0. 0. 0. 0. 0. 0.]
 [4. 0. 0. 0. 0. 0. 0.]
 [5. 0. 0. 0. 0. 0. 0.]
 [6. 0. 0. 0. 0. 0. 0.]
 [7. 0. 0. 0. 0. 0. 0.]
 [8. 0. 0. 0. 0. 0. 0.]]

For Saturday up to Sat: Got the following values:
[[0. 1. 2. 3. 4. 5. 6.]
 [1. 0. 1. 2. 3. 4. 5.]
 [2. 1. 1. 2. 3. 3. 4.]
 [3. 2. 2. 2. 3. 4. 4.]
 [4. 0. 0. 0. 0. 0. 0.]
 [5. 0. 0. 0. 0. 0. 0.]
 [6. 0. 0. 0. 0. 0. 0.]
 [7. 0. 0. 0. 0. 0. 0.]
 [8. 0. 0. 0. 0. 0. 0.]]

For Saturday up to Satu: Got the following values:
[[0. 1. 2. 3. 4. 5. 6.]
 [1. 0. 1. 2. 3. 4. 5.]
 [2. 1. 1. 2. 3. 3. 4.]
 [3. 2. 2. 2. 3. 4. 4.]
 [4. 3. 2. 3. 3. 4. 5.]
 [5. 0. 0. 0. 0. 0. 0.]
 [6

Finally, the final Levenshtein distance is given by the lower-right corner of the column, which can be accessed using the same notation:

In [23]:
print(d[len(str1)][len(str2)])

3.0


As a final exercise, can you implement the above into a function for two **arbitrary strings**? The outline is below:

In [ ]:
def lev_distance(str1, str2):
    #Your code here